# Extracting 'essential' gene protein sequences identified by van Opijnen (2009) from TIGR4 closed genome

In 2009, van Opijnen et al. (https://www.nature.com/articles/nmeth.1377) developed a method to determine gene essentiality in a high throughput way. They used S. pneumoniae as the model organism (TIGR4 strain, specifically) and identified 398 genes that were likely to be essential (necessary for survival). 346 of these genes were estimated essential with high certainty, 339 of these have high quality sequences in the genomic feature file used by van Opijnen, et al. AND appear consistently in our draft pneumococcal genomes.

For this, I first have to extract the locus tag for the essential genes identified in the van Opijnen et al. (2009) paper and then use those locus tags to extract the protein sequences from the TIGR4 closed genome. I have downloaded the genomic features file for the closed TIGR4 genome from NCBI (NCBI accession: GCF_000006885.1). This notebook generates a fasta file with the protein sequences for these essential genes, so that I can query them against the pneumococcal pangenome analyses.

### Import necessary packages

In [66]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

### Extract locus tags from van Opijnen et al. supplementary data

In [67]:
# Import data
opij_data = pd.read_excel('~/tnseq_essential_genes_paper_data.xls')
# Table ends at row 2086, but has some other annotations
opij_data_clean = opij_data.iloc[:2086 + 1]  

In [68]:
# Filter by genes identified as essential
essential_df = opij_data_clean[opij_data_clean['essential']=='E']

# Extract locus tags 
target_old_locus_tags = list(essential_df['locus'])

In [69]:
# Input GenBank file
input_gbff = "~/TIGR4_closed_ref_2001.gbff"

# Output FASTA file
output_fasta = "~/strep_pneumo_essential_protein_seqs.fasta"

with open(output_fasta, 'w') as out_fasta:
    for record in SeqIO.parse(input_gbff, "genbank"):
        for feature in record.features:
            if feature.type == "CDS":
                qualifiers = feature.qualifiers
                old_locus_tag = qualifiers.get('old_locus_tag', [None])[0]
                if old_locus_tag in target_old_locus_tags:
                    gene_name = qualifiers.get('gene', ['unnamed_gene'])[0]
                    translation = qualifiers.get('translation', [''])[0]
                    out_fasta.write(f'>{gene_name}-{old_locus_tag}\n{translation}\n')             

## Function to translate dna fasta file into protein fasta file

This translates the CLARC output nucleotide file into amino acid sequences, to that the essential genes can be queried.

In [61]:
def translate_dna_to_protein(input_fasta, output_fasta):
    with open(input_fasta, "r") as fasta_in, open(output_fasta, "w") as fasta_out:

        for record in SeqIO.parse(fasta_in, "fasta"):
            protein_seq = record.seq.translate()
            
            # Remove the stop codon (*) from the end of the protein sequence
            if protein_seq.endswith("*"):
                protein_seq = protein_seq[:-1]
                
            protein_record = SeqRecord(protein_seq, id=record.id, description=record.description)
            SeqIO.write(protein_record, fasta_out, "fasta")